In [1]:
%pip install fiona

     ---------------------------------------- 21.9/21.9 MB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import fiona
import csv
import os

In [5]:
class ShapefileValidator:
    
    def __init__(self, shapefile_path):
        self.shapefile_path = shapefile_path
        self.layer = fiona.open(self.shapefile_path)
        self.geom_types = ['Point', 'MultiPoint', 'LineString', 'MultiLineString', 'Polygon', 'MultiPolygon']
        
    def validate_features(self):
        for feature in self.layer:
            try:
                fiona.validate_feature(feature)
            except Exception as e:
                print(f"Invalid feature: {e}")
                
    def check_intersection(self):
        geom_type = self.layer.schema['geometry']
        if geom_type not in ['LineString', 'MultiLineString', 'Polygon', 'MultiPolygon']:
            print("Intersection check is not applicable for this geometry type.")
        else:
            features = [feat for feat in self.layer]
            for i, feat1 in enumerate(features):
                geom1 = feat1['geometry']
                for j, feat2 in enumerate(features[i+1:], i+1):
                    geom2 = feat2['geometry']
                    if geom1.intersects(geom2):
                        print(f"Features {i} and {j} intersect.")
                        
    def remove_invalid_geometry(self, output_path):
        with fiona.open(output_path, 'w', **self.layer.meta) as output:
            for feature in self.layer:
                try:
                    fiona.validate_feature(feature)
                    output.write(feature)
                except:
                    print(f"Invalid feature. Skipping: {feature}")
                    
    def remove_intersecting_geometry(self, output_path):
        geom_type = self.layer.schema['geometry']
        if geom_type not in ['LineString', 'MultiLineString', 'Polygon', 'MultiPolygon']:
            print("Intersection check is not applicable for this geometry type.")
        else:
            features = [feat for feat in self.layer]
            filtered_features = []
            for i, feat1 in enumerate(features):
                geom1 = feat1['geometry']
                intersecting = False
                for j, feat2 in enumerate(features[i+1:], i+1):
                    geom2 = feat2['geometry']
                    if geom1.intersects(geom2):
                        print(f"Features {i} and {j} intersect. Removing feature {j}.")
                        intersecting = True
                        break
                if not intersecting:
                    filtered_features.append(feat1)
            with fiona.open(output_path, 'w', **self.layer.meta) as output:
                for feature in filtered_features:
                    output.write(feature)
                    
    def to_csv(self, output_path):
        geom_type = self.layer.schema['geometry']
        if geom_type != 'Point':
            print("CSV conversion is only applicable for Point geometry.")
        else:
            with open(output_path, 'w', newline='') as output:
                writer = csv.writer(output)
                writer.writerow(['x', 'y'])
                for feature in self.layer:
                    point = feature['geometry']['coordinates']
                    writer.writerow(point)

    def run(self):
        shapefile_path = input("Enter shapefile path: ")
        if not os.path.exists(shapefile_path):
            print("File does not exist.")
            return
        
        self.sf = shapefile.Reader(shapefile_path)
        print("Options:\n1. Validate\n2. Check intersection\n3. Remove invalid geometry and export to shapefile\n4. Remove intersecting geometry and export to shapefile\n5. Convert to CSV")
        option = int(input("Enter option number: "))
        if option == 1:
            self.validate_features()
        elif option == 2:
            self.check_intersection()
        elif option == 3:
            filename, path = self.output_path
            self.remove_invalid_geometry(path, filename)
        elif option == 4:
            filename, path = self.output_path
            self.remove_intersecting_geometries(path, filename)
        elif option == 5:
            filename, path = self.output_path
            self.convert_to_csv(path, filename)
        else:
            print("Invalid option number.")
            

In [8]:
shp_validator = ShapefileValidator("C:/Users/polly/Documents/Modules/GIS 708 - Advanced GIS/Part 1 - Katumba/Practical 1/Practical 1_Dataset/PR_2011.shp")
shp_validator.run()

NameError: name 'os' is not defined